In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression, LassoCV, Lasso
from sklearn.metrics import roc_auc_score, brier_score_loss, log_loss
from xgboost import XGBClassifier
from group_lasso import GroupLasso
from collections import Counter

# Ideal

In [9]:
top = 20

lags = [21, 63]
responses = ['MI', 'Adj_MI', 'SI']
scales = ['daily', 'monthly']
targets = ['recession', 'score']
goals = ['explain', 'predict']
regressions = ['quantile', 'expectile']
selections = ['static', 'dynamic']
windows = ['63', '126']
sectors = ['Cap', 'Fin', 'Ele', 'FinEle']
variables = ['frm', 'lambda_j']
methods = ['logistic', 'asymmetric logistic', 'xgboost']
features = ['none', 'lasso', 'lasso_company', 'lasso_day']

# Reality

In [10]:
top = 20

lags = [21, 63]
responses = ['MI', 'Adj_MI', 'SI']
scales = ['daily', 'monthly']
targets = ['recession']
goals = ['explain', 'predict']
regressions = ['quantile']
selections = ['dynamic']
windows = ['63', '126']
sectors = ['Cap', 'Fin', 'Ele', 'FinEle']
variables = ['frm', 'lambda_j']
methods = ['logistic', 'asymmetric logistic', 'xgboost']
features = ['none', 'lasso', 'lasso_company', 'lasso_day']

column_titles = [
    'Sector', 'Window', 'Selection', 'Regression', 'Scale', 'Response', 'Lag', 'Variable', 'Target', 'Goal', 'Method', 'Feature',
    'In-AUC', 'In-Brier', 'In-LL', 'Out-AUC', 'Out-Brier', 'Out-LL'
]
if not os.path.isfile('Performance.csv'):
    df = pd.DataFrame(columns=column_titles)
    df.to_csv('Performance.csv', index=False)

In [11]:
def load_x_data(sector, scale, window, selection, lag, variable, input_path1, input_path2):
    full_lambda = pd.read_csv(f'{input_path1}/full_lambda.csv', index_col='Date', parse_dates=['Date'])

    if variable == 'frm':
        FRM = full_lambda.mean(axis=1) * full_lambda.shape[1] / top
        FRM.name = 'FRM'
        lagged_data = pd.DataFrame(FRM)
    elif variable == 'lambda_j':
        stock_rank = pd.read_csv(f'{input_path2}/Top rank company.csv', index_col='Date', parse_dates=['Date']).astype(str)
        lambda_rank = pd.DataFrame(index=full_lambda.index, columns=range(1, top + 1))
        for date in full_lambda.index:
            for col in lambda_rank.columns:
                company_id = stock_rank.loc[date, str(col)].split('.')[0]
                lambda_rank.loc[date, col] = full_lambda.loc[date, company_id]
        scaler = MinMaxScaler()
        lambda_rank[:] = scaler.fit_transform(lambda_rank.values)
        lagged_data = lambda_rank.copy()
    else:
        raise ValueError(f"Unsupported variable: {variable}")

    lagged_L = pd.DataFrame()

    if scale == 'daily':
        for column in lagged_data:
            lagged_cols = {f'{column}_{lag_ws}': lagged_data[column].shift(lag_ws) for lag_ws in range(lag + 1)}
            lagged = pd.concat(lagged_cols, axis=1).dropna()
            lagged_L = pd.concat([lagged_L, lagged], axis=1)
    elif scale == 'monthly':
        for column in lagged_data:
            lagged_cols = {f'{column}_{lag_ws}': lagged_data[column].shift(lag_ws) for lag_ws in range(lag + 1)}
            lagged = pd.concat(lagged_cols, axis=1).dropna().resample('M').last()
            lagged_L = pd.concat([lagged_L, lagged], axis=1)

    X = lagged_L.copy()
    return X
    

def load_y_data(response, scale, lag, target, goal):
    TAIEX = pd.read_csv('../01 Raw Data/Others/TAIEX.csv', index_col='Date', parse_dates=True)
    TAIEX_return = np.log(TAIEX['TAIEX'] / TAIEX['TAIEX'].shift(1)).dropna()
    trading_date = TAIEX['TAIEX'].copy()
    if response in ['MI', 'Adj_MI']:
        MI = pd.read_csv('../01 Raw Data/Others/MI.csv', index_col='Date', parse_dates=['Date'])
        MI = MI.resample('D').ffill()
        last_trading_date = trading_date.resample('M').apply(lambda x: x.index[-1])
        if scale == 'daily':
            if response == 'Adj_MI':
                MI_end_of_month = MI.reindex(last_trading_date.values)
                MI_filled = MI_end_of_month.reindex(TAIEX.index)
                MI_filled.bfill(inplace=True)
                for i in range(1, len(TAIEX)):
                    date = TAIEX.index[i]
                    if date not in last_trading_date.values:
                        MI_filled.iloc[i] = MI_filled.iloc[i - 1] * (1 + TAIEX_return.iloc[i].item())
                    else:
                        MI_filled.iloc[i] = MI.loc[date, 'MI']
                MI = MI_filled
            else:
                MI.reindex(last_trading_date.values).reindex(trading_date.index).bfill()
        elif scale == 'monthly':
            MI = MI.reindex(last_trading_date.values)
        if goal == 'predict':
            MI = MI.shift(-21) if scale == 'daily' else MI.shift(-1)
            MI = MI.dropna()
        y = MI['MI'].apply(lambda x: 1 if x < 17 else 0) if target == 'recession' else MI['MI']

    if response == 'SI':
        SI = TAIEX.copy()
        SI['TAIEX_smooth'] = SI['TAIEX'].rolling(window=21, min_periods=1).mean()
        SI['YoY'] = SI['TAIEX_smooth'].pct_change(periods=252) * 100
        SI = SI.dropna()
        SI['Recession'] = SI['YoY'].apply(lambda x: 1 if x < -12 else 0)
        last_trading_date = trading_date.resample('M').apply(lambda x: x.index[-1])
        if scale == 'daily':
            SI.reindex(last_trading_date.values).reindex(trading_date.index).bfill()
        elif scale == 'monthly':
            SI = SI.reindex(last_trading_date.values)
        if goal == 'predict':
            SI = SI.shift(-21)
            SI = SI.dropna()
        y = SI['Recession'] if target == 'recession' else SI['YoY']
    return y

def input_xy(X, y):
    y = y.loc[y.index.to_series().apply(lambda x: pd.Period(x, freq='M')).isin(X.index.to_period('M').unique())].copy()
    X = X.apply(pd.to_numeric, errors='coerce')
    return X, y

def cut_check(X, y, cut):
    n_samples = len(y)
    while cut > 1:
        split_size = n_samples // cut
        if all(y[i * split_size : (i + 1) * split_size].nunique() > 1 for i in range(cut)):
            break
        cut -= 1
    return cut

def feature_selection(variable, feature, lag, X_train, X_test, y_train, y_test):
    if variable == 'frm' or feature == 'none':
        pass
    else:
        if feature == 'lasso':
            lasso_cv = LassoCV(cv=10, random_state=42, max_iter=100000, n_jobs=-1).fit(X_train, y_train)
            best_alpha = lasso_cv.alpha_
            lasso = Lasso(alpha=best_alpha)
            lasso.fit(X_train, y_train)
            selected_features = np.where(lasso.coef_ != 0)[0]
            count = 0
            while len(selected_features) < top and count < 10:
                count += 1
                lasso = Lasso(alpha=best_alpha * (0.5 ** count))
                lasso.fit(X_train, y_train)
                selected_features = np.where(lasso.coef_ != 0)[0]
            
            X_train = X_train.iloc[:, selected_features]
            X_test = X_test.iloc[:, selected_features]

        if feature in ['lasso_company', 'lasso_day']:
            group = [k // (lag + 1) for k in range(X_train.shape[1])] if feature == 'lasso_company' else list(range(lag+1)) * top
            group_lasso = GroupLasso(groups=group, group_reg=0.01, l1_reg=0.01, scale_reg="group_size", n_iter=1000, tol=1e-3, supress_warning=True, random_state=42)
            group_lasso.fit(X_train, y_train)
            selected_features = np.where(group_lasso.coef_ != 0)[0]
            if len(selected_features) == 0:
                first_group = np.where(np.array(group) == 0)[0]
                selected_features = first_group

            X_train = X_train.iloc[:, selected_features]
            X_test = X_test.iloc[:, selected_features]
            if feature == 'lasso_company':
                coef_matrix = np.zeros((lag+1, top))
                for k in range(top):
                    coef_matrix[:, k] = group_lasso.coef_[k * (lag+1):(k+1) * (lag+1)].flatten()
                coef_df = pd.DataFrame(coef_matrix, columns=range(1, top+1), index=range(0, (lag+1)))
            elif feature == 'lasso_day':
                coef_matrix = np.zeros((top, lag+1))
                for k in range(lag+1):
                    coef_matrix[:, k] = group_lasso.coef_[k::(lag+1)].flatten()
                coef_df = pd.DataFrame(coef_matrix, columns=range(1, lag+1 + 1), index=range(0, (top)))
            else:
                raise ValueError(f"Unsupported configuration: {variable}-{feature}-{lag}")
    
    return X_train, X_test

def model(method, X_train, X_test, y_train, y_test):
    scaler = StandardScaler()
    X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)
    if method == 'logistic':
        model = LogisticRegression()
    elif method == 'asymmetric logistic':
        counter = Counter(y_train)
        samples = len(y_train)
        weight = {cls: samples/(2*count) for cls, count in counter.items()}
        model = LogisticRegression(class_weight=weight)
    elif method == 'xgboost':
        weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])
        model = XGBClassifier(scale_pos_weight=weight, random_state=42)
    else:
        raise ValueError(f"Unknown method: {method}")
    model.fit(X_train, y_train)
    return model

def performance(trained_model, X_train, X_test, y_train, y_test):
    y_train_pred = trained_model.predict(X_train)
    y_test_pred = trained_model.predict(X_test)
    y_train_proba = trained_model.predict_proba(X_train)[:, 1]
    y_test_proba = trained_model.predict_proba(X_test)[:, 1]

    in_auc = round(roc_auc_score(y_train, y_train_proba), 4)
    in_brier = round(brier_score_loss(y_train, y_train_proba), 4)
    in_ll = round(log_loss(y_train, y_train_proba), 4)
    
    out_auc = round(roc_auc_score(y_test, y_test_proba), 4)
    out_brier = round(brier_score_loss(y_test, y_test_proba), 4)
    out_ll = round(log_loss(y_test, y_test_proba), 4)

    return {'In-AUC': in_auc, 'In-Brier': in_brier, 'In-LL': in_ll, 'Out-AUC': out_auc, 'Out-Brier': out_brier, 'Out-LL': out_ll}

In [12]:
total_combinations = len(lags) * len(responses) * len(scales) * len(targets) * len(goals) * len(regressions) * len(selections) * len(windows) * len(sectors) * len(variables) * len(methods) * len(features)

with tqdm(total=total_combinations, desc="Processing combinations") as pbar:
    for sector in sectors:
        for window in windows:
            for selection in selections:
                for regression in regressions:
                    for scale in scales:
                        for response in responses:
                            for lag in lags:
                                for variable in variables:
                                    for target in targets:
                                        for goal in goals:
                                            for method in methods:
                                                for feature in features:
                                                    pbar.update(1)

                                                    existing_data = pd.read_csv('Performance.csv')
                                                    exists = existing_data[
                                                        (existing_data['Sector'] == sector) &
                                                        (existing_data['Window'] == window) &
                                                        (existing_data['Selection'] == selection) &
                                                        (existing_data['Regression'] == regression) &
                                                        (existing_data['Scale'] == scale) &
                                                        (existing_data['Response'] == response) &
                                                        (existing_data['Lag'] == lag) &
                                                        (existing_data['Variable'] == variable) &
                                                        (existing_data['Target'] == target) &
                                                        (existing_data['Goal'] == goal) &
                                                        (existing_data['Method'] == method) &
                                                        (existing_data['Feature'] == feature)
                                                    ].shape[0] > 0

                                                    if exists:
                                                        continue

                                                    try:
                                                        # Input the FRM data, the Original data.
                                                        input_path1 = f'../03 Modeling/{sector}_{window}'
                                                        input_path2 = f'../01 Raw Data/{sector}'

                                                        X = load_x_data(sector, scale, window, selection, lag, variable, input_path1, input_path2)
                                                        y = load_y_data(response, scale, lag, target, goal)
                                                        X, y = input_xy(X, y)

                                                        cut = cut_check(X, y, 4) if target == 'recession' else 4
                                                        n_samples = X.shape[0]
                                                        split_size = n_samples // cut

                                                        In_AUC_list, In_Brier_list, In_LL_list = [], [], []
                                                        Out_AUC_list, Out_Brier_list, Out_LL_list = [], [], []

                                                        for i in range(cut):
                                                            start_test = i * split_size
                                                            end_test = (i+1) * split_size if i < cut - 1 else n_samples

                                                            X_test = X[start_test:end_test]
                                                            y_test = y[start_test:end_test]

                                                            indices = list(range(0, start_test)) + list(range(end_test, n_samples))
                                                            X_train = X.iloc[indices]
                                                            y_train = y.iloc[indices]

                                                            scaler = StandardScaler()
                                                            X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X.columns)
                                                            X_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)

                                                            X_train, X_test = feature_selection(variable, feature, lag, X_train, X_test, y_train, y_test)

                                                            trained_model = model(method, X_train, X_test, y_train, y_test)
                                                            model_performance = performance(trained_model, X_train, X_test, y_train, y_test)

                                                            In_AUC_list.append(model_performance['In-AUC'])
                                                            In_Brier_list.append(model_performance['In-Brier'])
                                                            In_LL_list.append(model_performance['In-LL'])
                                                            Out_AUC_list.append(model_performance['Out-AUC'])
                                                            Out_Brier_list.append(model_performance['Out-Brier'])
                                                            Out_LL_list.append(model_performance['Out-LL'])

                                                        In_AUC, In_Brier, In_LL = map(lambda x: round(np.mean(x), 4), [In_AUC_list, In_Brier_list, In_LL_list])
                                                        Out_AUC, Out_Brier, Out_LL = map(lambda x: round(np.mean(x), 4), [Out_AUC_list, Out_Brier_list, Out_LL_list])
                                                        
                                                        result_df = pd.DataFrame([[
                                                            sector, window, selection, regression, scale, response, lag, variable, target, goal, method, feature,
                                                            In_AUC, In_Brier, In_LL, Out_AUC, Out_Brier, Out_LL
                                                        ]], columns=column_titles)

                                                        result_df.to_csv('Performance.csv', mode='a', header=False, index=False)

                                                    except Exception as e:
                                                        print(f"Error processing combination: {sector}, {window}, {selection}, {regression}, {scale}, {response}, {lag}, {variable}, {target}, {goal}, {method}, {feature} - {e}")
                                                        continue

Processing combinations:   2%|▏         | 73/4608 [06:42<1:43:11,  1.37s/it] /opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sc

Error processing combination: Cap, 63, dynamic, quantile, monthly, SI, 21, frm, recession, predict, logistic, none - positional indexers are out-of-bounds
Error processing combination: Cap, 63, dynamic, quantile, monthly, SI, 21, frm, recession, predict, logistic, lasso - positional indexers are out-of-bounds
Error processing combination: Cap, 63, dynamic, quantile, monthly, SI, 21, frm, recession, predict, logistic, lasso_company - positional indexers are out-of-bounds
Error processing combination: Cap, 63, dynamic, quantile, monthly, SI, 21, frm, recession, predict, logistic, lasso_day - positional indexers are out-of-bounds
Error processing combination: Cap, 63, dynamic, quantile, monthly, SI, 21, frm, recession, predict, asymmetric logistic, none - positional indexers are out-of-bounds
Error processing combination: Cap, 63, dynamic, quantile, monthly, SI, 21, frm, recession, predict, asymmetric logistic, lasso - positional indexers are out-of-bounds


Processing combinations:  11%|█         | 502/4608 [2:10:05<11:08,  6.14it/s]

Error processing combination: Cap, 63, dynamic, quantile, monthly, SI, 21, frm, recession, predict, asymmetric logistic, lasso_company - positional indexers are out-of-bounds
Error processing combination: Cap, 63, dynamic, quantile, monthly, SI, 21, frm, recession, predict, asymmetric logistic, lasso_day - positional indexers are out-of-bounds
Error processing combination: Cap, 63, dynamic, quantile, monthly, SI, 21, frm, recession, predict, xgboost, none - positional indexers are out-of-bounds
Error processing combination: Cap, 63, dynamic, quantile, monthly, SI, 21, frm, recession, predict, xgboost, lasso - positional indexers are out-of-bounds
Error processing combination: Cap, 63, dynamic, quantile, monthly, SI, 21, frm, recession, predict, xgboost, lasso_company - positional indexers are out-of-bounds
Error processing combination: Cap, 63, dynamic, quantile, monthly, SI, 21, frm, recession, predict, xgboost, lasso_day - positional indexers are out-of-bounds


Processing combinations:  11%|█         | 518/4608 [2:11:50<6:42:45,  5.91s/it] 

...

Error processing combination: FinEle, 126, dynamic, quantile, monthly, SI, 63, lambda_j, recession, predict, xgboost, lasso - positional indexers are out-of-bounds


Processing combinations: 100%|██████████| 4608/4608 [28:02:15<00:00,  2.66s/it]

Error processing combination: FinEle, 126, dynamic, quantile, monthly, SI, 63, lambda_j, recession, predict, xgboost, lasso_company - positional indexers are out-of-bounds


Processing combinations: 100%|██████████| 4608/4608 [28:02:17<00:00, 21.90s/it]

Error processing combination: FinEle, 126, dynamic, quantile, monthly, SI, 63, lambda_j, recession, predict, xgboost, lasso_day - positional indexers are out-of-bounds
